In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
import os
from mpl_toolkits.basemap import Basemap
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams['figure.figsize'] = (10,12)
from ipywidgets import interact

In [ ]:
bdir = 'C:/Users/Bruno Silveira/Documents/Anaconda files program/ECMWF' # Diretório dos arquivos

flist =[]

for yy in range (1997,2018):
    for mm in range(1,13):
        mmfl = sorted(glob(bdir + '/' + str(yy) + '/' + str(mm) + '/' + '*.nc')) # Ordenando os arquivos por data
        for fp in mmfl:
            flist.append(fp) # Adicionando os arquivos na lista 'flist'

In [ ]:
flist

In [ ]:
dsfiles = xr.open_mfdataset(flist, engine ='netcdf4', concat_dim= 'fst', chunks= {'time': 46}) # Leitura dos arquivos

In [ ]:
dsfiles

In [ ]:
pr = dsfiles['pr'] # Acessando a 'Precipitação'
prec_n = pr.sel(longitude=slice(-50, -30), latitude=slice(-20, 10)) # Selecionando a região do Nordeste
#prec = pr
prec_n

In [ ]:
longitude = prec_n['longitude'][:]
latitude = prec_n['latitude'][:]
time = prec_n['time'][:]
member = prec_n['member'][:]

In [ ]:
def plot_graphic(data, information, step_parallels, step_meridians, resolution_graphic, var):
    '''
    Função para plotagem de gráficos.\n
    Especificações dos parâmetros:\n
        data: Deve receber os dados que serão plotados, como por exemplo: 
            prec[0,[0,3,4]].mean(['member']) => Aqui iremos obter a Precipitação Média\n\t\tentre os Membros 0,3 e 4.\n
        information: Deve receber as informações a serem mostradas no título do gráfico.\n
        step_parallels: A forma como será feito o espaçamento entre as linhas paralelas.\n        
        step_meridians: A forma como será feito o espaçamento entre as linhas meridianas.\n
        var: Booleano que decidirá se o gráfico será salvo.
    '''
    
    m = Basemap(resolution = resolution_graphic, 
            llcrnrlon=longitude.min(),
            urcrnrlon=longitude.max(),
            llcrnrlat=latitude.min(),
            urcrnrlat=latitude.max())
    x, y = m(*np.meshgrid(longitude,latitude))
    m.pcolormesh(x,y,data,shading='flat',cmap=plt.cm.jet)
    m.colorbar(location='right', pad= '7%')
    m.drawcoastlines(linewidth = 1.7)
    m.drawstates(linewidth = 1.)
    m.drawparallels(np.arange(y.min(),y.max(),step_parallels),labels=[1,0,0,0])
    m.drawmeridians(np.arange(x.min(),x.max(),step_meridians),labels=[0,0,0,1])
    plt.xlabel('Longitude', labelpad = 30)
    plt.ylabel('Latitude', labelpad = 35)
    plt.title(information.upper(), pad = 15, linespacing = 1.3)
    plt.rcParams['figure.figsize'] = (10,12)
    
    
    if var == True:
        number_figure = input('Digite o número da figura:\n')
        plt.savefig('figure{}.png'.format(number_figure))

In [ ]:
plot_graphic(prec_n[3,:7,:].sum('time').mean('member'),
             'média entre todos os membros: Acumulado de 7 dias\n (1997-10-02T12:00:00 -- 1997-10-08T12:00:00)'
             , 2, 2,'i', False)

In [ ]:
plot_graphic(prec_n[0,:7,:].sum('time').std('member'),
             'devio padrão entre todos os membros: Acumulado de 7 dias\n (1997-10-02T12:00:00 -- 1997-10-08T12:00:00)'
             , 2, 2,'i', False)